In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
input_folder = r"D:\AShare_test"
csv_files = [f for f in os.listdir(input_folder) if f.endswith(".csv")]

In [3]:
features = ['return1','open','close','high','low','volume','VWAP','turn','free_turn']

In [35]:
file_path = os.path.join(input_folder, 'bj430017.csv')
df = pd.read_csv(file_path)

In [36]:
n = 30
for feature in features:
    df[feature] = (df[feature] - df[feature].rolling(n).mean().shift(1)) / (df[feature].rolling(n).std().shift(1) + 1e-8)

In [37]:
df

,code,date,return1,open,close,high,low,volume,VWAP,turn,free_turn
0,bj430017,2023-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,bj430017,2023-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,bj430017,2023-06-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bj430017,2023-06-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,bj430017,2023-06-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
382,bj430017,2024-12-25,0.064017,-1.752495,-1.995929,-1.815942,-1.740122,-1.060820,-1.830109,-1.059533,-1.059232
383,bj430017,2024-12-26,0.004822,-1.944248,-1.917166,-2.051749,-1.886836,-0.917077,-1.960288,-0.915637,-0.915344
384,bj430017,2024-12-27,0.334912,-1.836435,-1.694861,-1.861852,-1.598697,-1.319092,-1.758422,-1.319302,-1.319142
385,bj430017,2024-12-30,-0.979916,-1.584324,-1.938404,-1.788278,-1.813155,-1.577424,-1.826551,-1.578802,-1.578748


In [5]:
X, Y, Y_dates, empty = [], [], [], []
def get_window(file_name):
    file_path = os.path.join(input_folder, file_name)
    df = pd.read_csv(file_path)
    x, y, dates = [], [], []
    n = 30
    
    #for feature in features:
        #df[feature] = (df[feature] - df[feature].rolling(n).mean().shift(1)) / (df[feature].rolling(n).std().shift(1) + 1e-8)

    i = 0
    
    #每10个交易日进行一次数据回看取样
    while i + 40 < len(df):
        
        #当前的图片对应的日期
        date = df.iloc[i+40]['date']
        
        #除了date以外的数据形成图片，转制后传入x
        window = df.iloc[i:i+30, 1:]
        window.set_index('date', inplace=True)
        window = window.transpose().to_numpy()
        return_40 = df.iloc[i+40]['return1']
        #检查window和return_40是否为NaN
        if not np.isnan(window).any() and not np.isnan(return_40):         
            x.append(window)
            y.append(return_40)
            dates.append(date)
        
        i += 10
        
    if len(x) > 0:
        x = np.stack(x) #二维数据堆叠为三维数据，假如一共有K个图片，则x形状为（K，9，30）
        y = np.stack(y)
        y_dates = np.stack(dates)

        X.append(x) #x仅为当前股票的数据，X集合为所有股票的数据
        Y.append(y)
        Y_dates.append(y_dates)
    else:
        empty.append(df['code'][0])

In [8]:
for file in tqdm(csv_files, total=len(csv_files)):
    get_window(file)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:01<00:00,  4.71it/s]


In [ ]:
Y_dates = np.concatenate(Y_dates, axis=0) #将所有股票日期沿第一个维度拼接（可能会有重复）
order = np.argsort(Y_dates) #获得Y_dates排序后的索引，并且是稳定排序stable sort
X = np.concatenate(X, axis=0)[order]
Y = np.concatenate(Y, axis=0)[order]
Y_dates = Y_dates[order]

print('Shape of X: ', X.shape)
print('Shape of Y: ', Y.shape)
print('Stocks with not enough data: ', empty)

In [ ]:
np.save('X_fe.npy', X)
np.save('Y_fe.npy', Y)
np.save('Y_dates.npy', Y_dates)